In [103]:
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from dotenv import load_dotenv
from langsmith import Client
import json

import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

load_dotenv() 
client = Client() # langsmith 추적

class RealEstateState(TypedDict): # 그래프의 상태를 정의하는 클래스
    real_estate_type: Annotated[str ,"부동산 유형 (예: 아파트, 상가)"]
    keywordlist: Annotated[List[Dict] ,"키워드 리스트"]
    messages: Annotated[list, add_messages]
    query_sql: Annotated[str ,"생성된 SQL 쿼리"]
    results: Annotated[List[Dict], "쿼리 결과"]
    answers: Annotated[List[str], "최종 답변 결과"]
    query_answer:Annotated[str, 'answer다듬기']

# Step 1: LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

# Step 2: StateGraph 정의
workflow = StateGraph(RealEstateState)

In [104]:
def filter_node(state:RealEstateState) -> RealEstateState:
    system_prompt = """
    Classify if a given question is related to real estate. If the question is related to topics such as property transactions, rental conditions, location recommendations, or property features, return Pass. If it's not directly related to real estate, return Fail.

    # Output Format
    - Return Pass if the question is real estate-related, otherwise return Fail.

    # Examples
    - 입력: '서울 아파트 매매 가격이 어떻게 되나요?'
      출력: Pass

    - 입력: '이 동네 전세 시세 알려주세요'
      출력: Pass

    - 입력: '서울에서 월세 계약 조건이 어떻게 되나요?'
      출력: Pass

    - 입력: '서울 아파트 매매가 얼마인가요?'
      출력: Pass

    - 입력: '대치동에서 버스정류장과 지하철이 가장 가까운 곳으로 알려줘'
      출력: Pass

    - 입력: '여자 혼자 살기 좋은 곳 추천해줘'
      출력: Pass

    - 입력: '교통이 편리하고 저렴한 원룸 추천해줘'
      출력: Pass

    - 입력: '이 음식점이 맛있나요?'
      출력: Fail

    - 입력: '서울에서 가장 큰 공원이 어디인가요?'
      출력: Fail
    """
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(state["messages"][-1].content)
    ])

    real_estate_type = response.content.strip()
    return RealEstateState(real_estate_type=real_estate_type)

def fiter_router(state: RealEstateState):
    # This is the router
    real_estate_type = state["real_estate_type"]
    if real_estate_type == "Pass":
        return "Pass"
    else:
        return 'Fail'
    
def re_questions(state: RealEstateState) -> RealEstateState:
    print("=================================")
    print("""[re_questions] 질문이 부동산 관련이 아니거나 제대로 인식되지 않았습니다.
          부동산 관련 질문을 좀 더 자세하게 작성해주시면 답변드리겠습니다!!!""")
    new_question = input("새로운 부동산 질문을 입력해주세요: ")
    print("=================================")
    # 수정된 질문을 state에 업데이트
    return RealEstateState(messages=new_question)

In [105]:
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

def get_db_engine(db_path):
    """로컬 SQLite DB 파일과 연결된 엔진을 생성합니다."""
    try:
        # SQLite DB 파일과 연결
        connection = sqlite3.connect(db_path, check_same_thread=False)
        # SQLAlchemy 엔진 생성
        engine = create_engine(
            f"sqlite:///{db_path}",
            poolclass=StaticPool,
            connect_args={"check_same_thread": False}
        )
        return engine
    except Exception as e:
        print(f"데이터베이스 연결 중 오류 발생: {str(e)}")
        return None
    
    
# DB 파일 경로 지정
db_path = './data/real_estate_(1).db'
engine = get_db_engine(db_path)
db = SQLDatabase(
    engine,
    sample_rows_in_table_info=False  # 샘플 행 조회 비활성화
)

In [106]:
def generate_query(state: RealEstateState) -> RealEstateState:
    table = db.get_table_info(table_names=["addresses","sales", "rentals", "property_info", "property_locations","location_distances", "subway_stations"])
    prompt = f"""
    다음 데이터베이스 구조를 기반으로 사용자의 질문에 대한 SQL 쿼리를 생성해주세요:
    
    테이블 및 주요 컬럼 설명:
    {table}

    거리 및 대중교통 관련 계산 규칙:
    - **대중교통과 매물간의 거리 계산**: 테이블들을 활용해서 계산합니다.

    주의사항:
    1. 관련 조건은 SQL WHERE 절 형식으로 작성해주세요.
    2. '최근 5년' 같은 상대적 시간 표현은 현재 날짜 기준으로 변환해주세요.
    3. '추천할 만한', '인기 있는' 등의 표현은 적절히 해석하여 조건을 추가해주세요. 예: 'crime_rate < 5' 또는 'population_level = "높음"'.
    4. 텍스트 검색에는 LIKE 또는 MATCH ... AGAINST를 활용하세요.
    5. 여러 조건은 AND 또는 OR로 연결하세요.
    6. 정렬, 그룹화 등이 필요한 경우 이를 추가로 명시하세요. 예: ORDER BY created_at DESC.
    7. 쿼리만 작성하고 추가 설명은 하지 마세요.
    8. rental_type 값은 다음과 같습니다:
        - 'MONTHLY': 전세
        - 'YEARLY': 월세
    9. facilities는 영어로 쳐야지 나옵니다.
        - pi.facilities LIKE '%ELEVATOR%' -> 엘리베이터
        - pi.facilities LIKE '%aircon%' -> 에이컨, 에어콘
    10. 매물번호도 함께 보여주세요 매물번호는 property_id입니다.
    11. rentals 테이블에 price라는 컬럼이 없습니다다. rentals 테이블에는 price 대신 deposit(보증금)과 monthly_rent(월세) 컬럼이 존재합니다.

    사용자 질문: {state['messages'][-1].content}

    SQL 쿼리 형식:
    SELECT * FROM table_name WHERE condition1 AND condition2 ...;
    """

    response = llm.invoke([
            SystemMessage(content="당신은 SQLite Database  쿼리를 생성하는 전문가입니다."),
            HumanMessage(prompt)
        ])
    
    return RealEstateState(query_sql=response.content)

In [107]:
def clean_sql_response(state: RealEstateState) -> RealEstateState:
    # 'query_sql' 키는 항상 존재한다고 가정
    query_sql = state['query_sql']

    # 코드 블록(````sql ... `````) 제거
    if query_sql.startswith("```sql") and query_sql.endswith("```"):
        query_sql = query_sql[6:-3].strip()  # "```sql" 제거 후 앞뒤 공백 제거

    # SQL 문 끝에 세미콜론 추가 (필요시)
    if not query_sql.strip().endswith(";"):
        query_sql = query_sql.strip() + ";"
        

    # 상태 업데이트
    return RealEstateState(query_sql=query_sql)

In [108]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

def run_query(state: RealEstateState) -> RealEstateState:
    
    tool = QuerySQLDataBaseTool(db=db)
    results = tool._run(state["query_sql"])

    return RealEstateState(results=results)

In [109]:
def generate_response(state: RealEstateState)-> RealEstateState:
    prompt = state['messages'][-1].content
    context = state['results']
    context += "위 매물들에 대한 추천과 함께, 각 부동산과 관련된 세계 지식과 흥미로운 사실을 추가해주세요."

    response = llm.invoke([
            SystemMessage(content="당신은 부동산산 추천 전문가이자 세계 지식을 갖춘 AI입니다. 주어진 정보와 세계 지식을 결합하여 사용자의 질문에 답변해주세요. 구분선 이후 간단한 추천이유도 적어줍니다."),
            HumanMessage(content= f"컨텍스트: {context}\n\n질문: {prompt}\n\n각 부동산에 대해 관련된 세계 지식과 흥미로운 사실을 추가해주세요.")
        ])
    
    output = response.content.strip()

    return RealEstateState(answers=output)

In [110]:
workflow.add_node("Filter Question", filter_node)
workflow.add_node('Re_Questions', re_questions)
workflow.add_node('Generate_Query', generate_query)
workflow.add_node('Clean_Sql_Response', clean_sql_response)
workflow.add_node('Run_Query', run_query)
workflow.add_node('Generate_Response', generate_response)

In [111]:
workflow.add_conditional_edges(
    "Filter Question",
    fiter_router,
    { 'Pass': "Generate_Query", 'Fail': 'Re_Questions'}
)

In [112]:
workflow.add_edge(START, "Filter Question")
workflow.add_edge("Re_Questions", "Filter Question")
workflow.add_edge("Generate_Query", "Clean_Sql_Response")
workflow.add_edge("Clean_Sql_Response", "Run_Query")
workflow.add_edge("Run_Query", "Generate_Response")
workflow.add_edge("Generate_Response", END)

In [113]:
app = workflow.compile()

In [100]:
result = app.invoke({'messages': '서울시 강남역에서 1000미터 이내 전세 10억 매물을 추천해줘'})

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000021BF1D6E260>>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000021BF1D6E260>>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [101]:
print(result['answers'])

주어진 정보에 따르면, 서울시 강북구에서 엘리베이터가 있는 전세 3억 매물 목록에는 해당하는 매물이 없어 보입니다. 하지만 비슷한 가격대의 매물을 추천드리고, 각 매물에 대해 관련된 세계 지식을 추가하겠습니다.

1. **드림캐슬 502호 (208,000,000원)**
   - **추천이유**: 강북구에서 잘 관리된 아파트 단지로, 엘리베이터가 설치되어 있어 편리합니다. 
   - **세계 지식**: '드림캐슬'은 현대 아파트 이름으로 "꿈의 성"이라는 뜻을 가지고 있습니다. 아파트는 현대 사회에서 많은 사람들이 살고 있는 주거 형태로, 높은 밀도의 생활 환경에서 편리한 접근성을 제공합니다.

2. **런던빌 501호 (255,000,000원)**
   - **추천이유**: 인기 있는 지역으로, 엘리베이터와 넓은 공간을 제공하는 매물입니다.
   - **세계 지식**: '런던빌'이라는 이름은 유명 도시 런던을 떠올리게 합니다. 런던은 다양한 문화와 역사를 자랑하는 도시로, 그 지역의 부동산은 투자 가치가 높습니다. 아파트 이름에서 도시 이름을 사용하는 것은 자칫 부동산의 이미지와 가치를 높이는 효과를 가져옵니다.

3. **드림캐슬 503호 (280,000,000원)**
   - **추천이유**: 높은 가격대지만, 위치와 시설이 잘 갖춰져 있어 가치가 높습니다.
   - **세계 지식**: 아파트 이름에서 "드림"이라는 단어를 사용하는 것은 심리적으로 긍정적인 이미지를 주며, 이는 소비자 선택에 큰 영향을 미칠 수 있습니다. 

4. **골드하우스 403호 (140,000,000원)**
   - **추천이유**: 가격은 저렴하지만, 엘리베이터를 갖춘 매물로서 가성비가 뛰어납니다.
   - **세계 지식**: "골드"라는 수식어는 일반적으로 럭셔리나 가치를 의미합니다. 골드라는 자원은 역사적으로 시장 가치가 높아, 골드하우스 또한 상승 가치를 지닐 수 있음을 나타냅니다.

대상이 되는 매물들은 가격대나 위치에서 약간의 차이가 있을 수 있지만, 엘리베이터의 유무와

In [102]:
print(result['results'])

[(6797780, None, '제1동 3층일부', 100000000, 0), (6805789, None, None, 290000000, 0), (6811503, None, '101호', 190000000, 0), (6813699, None, '1층일부', 40000000, 0), (6832825, None, '803호', 90000000, 0), (6836605, '드림캐슬', '503호', 280000000, 0), (6836610, '드림캐슬', '502', 208000000, 0), (6864208, None, None, 160000000, 0), (6876704, None, '302호', 230000000, 0), (6902487, None, '203  ,468-229', 200000000, 0), (6925185, None, '305호', 50000000, 0), (6934707, '런던빌', '런던빌 제에이동 제5층 501호', 255000000, 0), (6949204, None, '405호', 50000000, 0), (6950614, None, None, 299000000, 0), (6961512, None, '4층일부', 55000000, 0), (6969698, None, '301호', 250000000, 0), (6987072, '골드하우스', '골드화우스 403호', 140000000, 0)]
